In [81]:
import tweepy
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p

In [64]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [3]:
#Twitter credentials for the app
consumer_key = ''
consumer_secret = ''
access_key= 's'
access_secret = ''

In [6]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [41]:
covid19_tweets = "../Data/covid19/covid19_data.csv"

In [9]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [110]:
# A AMELIORER
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    
    #after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)    
    #Others clean
    tweet = tweet.lower() # convert text to lower-case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
    filtered_tweet = []

    #looping through conditions
    for w in tweet:
    #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)

In [39]:
def searchAndWrite_tweets(keyword, date_since, lang, count):
    tweets = tweepy.Cursor(api.search,
                        q=keyword,
                        lang=lang,
                        since=date_since).items(count)
    tweets_arr = []
    # Iterate and print tweets
    for tweet in tweets:
        tweets_arr.append(tweet._json)
    
    with open("../Data/covid19/data.json", "w+") as output:
        output.write(json.dumps(tweets_arr))
    return tweets_arr

In [29]:
search_words = ["#coronavirus", "#COVID19", "#CoronavirusOutbreak"]
date_since="2020-05-01"

In [40]:
tweets_arr = searchAndWrite_tweets(keyword=search_words, date_since=date_since, lang="en", count=200)

In [53]:
a = [[tweet['user']['screen_name'],tweet['created_at']] for tweet in tweets_arr]

In [130]:
tweet_details = [[tweet['id'], tweet['created_at'], tweet['text'], clean_tweets(p.api.clean(tweet['text'])),
                  tweet['user']['screen_name'], 
                  tweet['user']['followers_count'], tweet['user']['friends_count'], tweet['retweeted'], 
                  tweet['retweet_count'], tweet['user']['location']] for tweet in tweets_arr]

In [131]:
tweets_df = pd.DataFrame(
    data=tweet_details, 
    columns=['id', 'created_at', 'text', 'text_clean', 'screen_name', 
             'user_followers_count', 'user_friends_count', 'isRetweeted', 'retweet_count', 'user_location' ]
)

In [132]:
# Create a function to get the subjectivity
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity


In [133]:
# Create two new columns 'Subjectivity' & 'Polarity'
tweets_df['Subjectivity'] = tweets_df['text_clean'].apply(getSubjectivity)
tweets_df['Polarity'] = tweets_df['text_clean'].apply(getPolarity)